In [34]:
import os
import time
import pandas as pd
import pymysql
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm


def dev_db_conn():
    conn = pymysql.connect( 
        host = 호스트, 
        user = 유저, 
        password = 비밀번호, 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")
download_path = f"{file_path}사후면세점_이미지/"

# API 키 입력
tour_api_key = TOUR_API_KEY

# 작업 데이터 불러오기
free_tax_shopping = pd.read_excel(f"{file_path}사후면세점.xlsx", sheet_name="사후면세점_최종")
free_tax_shopping = free_tax_shopping.astype({"store_id":"int", "RSTR_ID":"int", "contentid":"int"})

In [ ]:
data_list = []
except_list = []

for i in tqdm(range(len(free_tax_shopping["contentid"]))):
    try:
        content_id = free_tax_shopping["contentid"][i]
        time.sleep(10)
        url = f"http://apis.data.go.kr/B551011/KorService1/detailImage1?ServiceKey={tour_api_key}&contentId=2924134&MobileOS=ETC&MobileApp=AppTest&imageYN=Y&subImageYN=Y&numOfRows=10"
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")
        item = soup.find("item")
        if item is not None:
            data_list.append({
                "store_id": free_tax_shopping["store_id"][i],
                "contentid": item.contentid.get_text() if item.contentid else None,
                "originimgurl": item.originimgurl.get_text() if item.originimgurl else None,
                "imgname": item.imgname.get_text() if item.imgname else None,
                "smallimageurl": item.smallimageurl.get_text() if item.smallimageurl else None,
                "cpyrhtdivcd": item.cpyrhtdivcd.get_text() if item.cpyrhtdivcd else None,
                "serialnum": item.serialnum.get_text() if item.serialnum else None
            })

        else:
            except_list.append(content_id)
    except:
        except_list.append(content_id)
    
df = pd.DataFrame(data_list, columns=["store_id", "contentid", "originimgurl", "imgname", "smallimageurl", "cpyrhtdivcd", "serialnum"])